test

In [ ]:
test

In [ ]:
library(colocboost)

In [ ]:
# example region
all_genes_path <- "/home/klawren/oak/gtex/output/coloc/Adipose_Subcutaneous/Adipose_Subcutaneous.chr1:16104-1170341.all_genes.colocboost.rds"
v39_genes_path <- "/home/klawren/oak/gtex/output/coloc/Adipose_Subcutaneous/Adipose_Subcutaneous.chr1:16104-1170341.v39_genes.colocboost.rds"
# cos2:y23_y82	is a protien coding that only exists with new lncRNA!
# cos9:y1_y61_y97 is two new lncRNA with protein coding gene

all_genes_res <- readRDS(all_genes_path)
v39_genes_res <- readRDS(v39_genes_path)

In [ ]:
png("all_genes_colocboost_plot.png", width=2000, height=2000)
colocboost_plot(all_genes_res[[1]], plot_ucos = TRUE)
dev.off()

png("v39_genes_colocboost_plot.png", width=2000, height=2000)
colocboost_plot(v39_genes_res[[1]], plot_ucos = TRUE)
# colocboost_plot(v39_genes_res[[1]], plot_ucos = TRUE, plot_all_outcome=TRUE)
dev.off()

agg_record_1708684070 
                    2

agg_record_1708684070 
                    2

In [ ]:
# Function to extract credible sets information with -log10(p-value) calculation
extract_credible_sets <- function(colocboost_result) {
  # Initialize empty lists to store results
  trait_specific_cs <- list()
  trait_shared_cs <- list()
  
  # Extract trait-specific credible sets (uCoS)
  if (!is.null(colocboost_result$ucos_details)) {
    ucos_variables <- colocboost_result$ucos_details$ucos$ucos_variables
    ucos_outcomes <- colocboost_result$ucos_details$ucos_outcomes$outcome_index
    ucos_weights <- colocboost_result$ucos_details$ucos_weight
    
    for (i in seq_along(ucos_variables)) {
      cs_name <- names(ucos_variables)[i]
      variants <- ucos_variables[[i]]
      outcome_idx <- ucos_outcomes[[i]]
      
      # Extract phenotype names
      phenotype_names <- colocboost_result$data_info$outcome_info$outcome_names
      phenotype_id <- phenotype_names[outcome_idx]
      
      # Get all variant names to match PIP values
      all_variants <- colocboost_result$data_info$variables
      
      # Extract PIP values for these specific variants
      variant_indices <- match(variants, all_variants)
      pip_values <- ucos_weights[[i]][variant_indices]
      
      # Extract z-scores and calculate -log10(p-values)
      z_scores <- colocboost_result$data_info$z[[outcome_idx]][variant_indices]
      neg_log10_p_values <- -log10(2 * pnorm(-abs(z_scores)))
      
      # Create dataframe for this credible set
      cs_df <- data.frame(
        phenotype_id = rep(phenotype_id, length(variants)),
        variant_id = variants,
        pip = pip_values,
        neg_log10_p_value = neg_log10_p_values,
        cs_id = rep(cs_name, length(variants)),
        cs_type = "trait_specific",
        stringsAsFactors = FALSE
      )
      
      trait_specific_cs[[i]] <- cs_df
    }
  }
  
  # Extract trait-shared credible sets (CoS)
  if (!is.null(colocboost_result$cos_details)) {
    cos_variables <- colocboost_result$cos_details$cos$cos_variables
    cos_outcomes <- colocboost_result$cos_details$cos_outcomes$outcome_index
    cos_weights <- colocboost_result$cos_details$cos_vcp
    
    for (i in seq_along(cos_variables)) {
      cs_name <- names(cos_variables)[i]
      variants <- cos_variables[[i]]
      outcome_indices <- cos_outcomes[[i]]
      
      # Extract phenotype names for all outcomes in this CoS
      phenotype_names <- colocboost_result$data_info$outcome_info$outcome_names
      phenotype_ids <- phenotype_names[outcome_indices]
      
      # Get all variant names to match PIP values
      all_variants <- colocboost_result$data_info$variables
      
      # Extract PIP values for these specific variants
      variant_indices <- match(variants, all_variants)
      pip_values <- cos_weights[[i]][variant_indices]
      
      # For each variant, get -log10(p-value) for each outcome
      variant_neg_log10_p_values_list <- list()
      for (v in seq_along(variants)) {
        variant_neg_log10_p_values <- c()
        for (j in seq_along(outcome_indices)) {
          outcome_idx <- outcome_indices[j]
          z_scores <- colocboost_result$data_info$z[[outcome_idx]][variant_indices]
          neg_log10_p_values <- -log10(2 * pnorm(-abs(z_scores)))
          variant_neg_log10_p_values <- c(variant_neg_log10_p_values, neg_log10_p_values[v])
        }
        variant_neg_log10_p_values_list[[v]] <- variant_neg_log10_p_values
      }
      
      # Create dataframe for this credible set
      cs_df <- data.frame(
        phenotype_id = rep(paste(phenotype_ids, collapse = ","), length(variants)),
        variant_id = variants,
        pip = pip_values,
        neg_log10_p_value = I(variant_neg_log10_p_values_list),
        cs_id = rep(cs_name, length(variants)),
        cs_type = "trait_shared",
        stringsAsFactors = FALSE
      )
      
      trait_shared_cs[[i]] <- cs_df
    }
  }
  
  # Combine all credible sets
  all_cs <- c(trait_specific_cs, trait_shared_cs)
  
  if (length(all_cs) > 0) {
    result_df <- do.call(rbind, all_cs)
    return(result_df)
  } else {
    return(data.frame(
      phenotype_id = character(0),
      variant_id = character(0),
      pip = numeric(0),
      neg_log10_p_value = list(),
      cs_id = character(0),
      cs_type = character(0),
      stringsAsFactors = FALSE
    ))
  }
}

# Extract credible sets from all_genes results (using the xqtl_coloc component)
all_genes_colocboost <- all_genes_res[[1]]
all_genes_cs <- extract_credible_sets(all_genes_colocboost)

print("All genes credible sets:")
print(head(all_genes_cs))
print(paste("Total credible sets:", length(unique(all_genes_cs$cs_id))))
print(paste("Trait-specific credible sets (uCoS):", length(unique(all_genes_cs$cs_id[all_genes_cs$cs_type == "trait_specific"]))))
print(paste("Trait-shared credible sets (CoS):", length(unique(all_genes_cs$cs_id[all_genes_cs$cs_type == "trait_shared"]))))
# Extract credible sets from v39_genes results  
v39_genes_colocboost <- v39_genes_res[[1]]
v39_genes_cs <- extract_credible_sets(v39_genes_colocboost)

print("\nV39 genes credible sets:")
print(head(v39_genes_cs))
print(paste("Total credible sets:", length(unique(v39_genes_cs$cs_id))))
print(paste("Trait-specific credible sets (uCoS):", length(unique(v39_genes_cs$cs_id[v39_genes_cs$cs_type == "trait_specific"]))))
print(paste("Trait-shared credible sets (CoS):", length(unique(v39_genes_cs$cs_id[v39_genes_cs$cs_type == "trait_shared"]))))


[1] "All genes credible sets:"
                             phenotype_id           variant_id        pip
1  Adipose_Subcutaneous.ENSG00000229376.3  chr1:806470:C:T:b38 0.75744020
2  Adipose_Subcutaneous.ENSG00000229376.3  chr1:804185:G:C:b38 0.05118255
3  Adipose_Subcutaneous.ENSG00000229376.3  chr1:804863:T:C:b38 0.05118255
4  Adipose_Subcutaneous.ENSG00000229376.3  chr1:805036:A:G:b38 0.05118255
5  Adipose_Subcutaneous.ENSG00000229376.3  chr1:771265:A:C:b38 0.04693517
6 Adipose_Subcutaneous.ENSG00000131591.18 chr1:1091327:C:A:b38 0.99806643
  neg_log10_p_value      cs_id        cs_type
1          8.402177 ucos11:y17 trait_specific
2           7.32258 ucos11:y17 trait_specific
3           7.32258 ucos11:y17 trait_specific
4           7.32258 ucos11:y17 trait_specific
5           7.07342 ucos11:y17 trait_specific
6          23.48059 ucos20:y41 trait_specific
[1] "Total credible sets: 18"
[1] "Trait-specific credible sets (uCoS): 2"
[1] "Trait-shared credible sets (CoS): 16"
[1] "\nV39 

In [ ]:
# Subset all_genes_cs to exclude credible sets where ANY variant appears in v39_genes_cs
# Get all variants from v39_genes_cs
v39_variants <- unique(v39_genes_cs$variant_id)
print(paste("Number of unique variants in v39_genes_cs:", length(v39_variants)))

# Get all unique credible set IDs from all_genes_cs
all_genes_cs_ids <- unique(all_genes_cs$cs_id)
print(paste("Number of unique credible sets in all_genes_cs:", length(all_genes_cs_ids)))

# For each credible set in all_genes_cs, check if ANY of its variants are in v39_genes_cs
credible_sets_to_exclude <- c()

for (cs_id in all_genes_cs_ids) {
  # Get all variants for this credible set
  cs_variants <- all_genes_cs$variant_id[all_genes_cs$cs_id == cs_id]
  
  # Check if any of these variants are in v39_genes_cs
  if (any(cs_variants %in% v39_variants)) {
    credible_sets_to_exclude <- c(credible_sets_to_exclude, cs_id)
  }
}

print(paste("Number of credible sets to exclude:", length(credible_sets_to_exclude)))
print("Credible sets to exclude:")
print(credible_sets_to_exclude)

# Filter all_genes_cs to exclude credible sets that have any variants in v39_genes_cs
all_genes_cs_filtered <- all_genes_cs[!all_genes_cs$cs_id %in% credible_sets_to_exclude, ]

print(paste("Number of rows in filtered all_genes_cs:", nrow(all_genes_cs_filtered)))
print(paste("Number of unique credible sets in filtered data:", length(unique(all_genes_cs_filtered$cs_id))))
print(paste("Number of trait-specific credible sets (uCoS) in filtered data:", 
            length(unique(all_genes_cs_filtered$cs_id[all_genes_cs_filtered$cs_type == "trait_specific"]))))
print(paste("Number of trait-shared credible sets (CoS) in filtered data:", 
            length(unique(all_genes_cs_filtered$cs_id[all_genes_cs_filtered$cs_type == "trait_shared"]))))

# Show the filtered results
print("\nFiltered credible sets:")
print(unique(all_genes_cs_filtered$cs_id))

all_genes_cs_filtered


[1] "Number of unique variants in v39_genes_cs: 87"
[1] "Number of unique credible sets in all_genes_cs: 18"
[1] "Number of credible sets to exclude: 11"
[1] "Credible sets to exclude:"
 [1] "ucos11:y17"           "ucos20:y41"           "cos1:y70_y71"        
 [4] "cos3:y7_y8"           "cos4:y5_y7_y8_y20"    "cos5:y37_y51"        
 [7] "cos6:y18_y19_y20_y38" "cos8:y43_y62_y73"     "cos10:y20_y70_y71"   
[10] "cos13:y27_y28_y29"    "cos14:y28_y29"       
[1] "Number of rows in filtered all_genes_cs: 15"
[1] "Number of unique credible sets in filtered data: 7"
[1] "Number of trait-specific credible sets (uCoS) in filtered data: 0"
[1] "Number of trait-shared credible sets (CoS) in filtered data: 7"
[1] "\nFiltered credible sets:"
[1] "cos2:y23_y82"       "cos7:y31_y34_y37"   "cos9:y1_y61_y97"   
[4] "cos11:y19_y102"     "cos12:y35_y36"      "cos15:y3_y4:merged"
[7] "cos16:y3_y4:merged"


,phenotype_id,variant_id,pip,neg_log10_p_value,cs_id,cs_type
,<chr>,<chr>,<dbl>,<list>,<chr>,<chr>
8,"Adipose_Subcutaneous.ENSG00000297224.1,Adipose_Subcutaneous.ENSG00000228594.4",chr1:1134712:G:T:b38,0.97711699,"4.279781, 4.822464",cos2:y23_y82,trait_shared
59,"Adipose_Subcutaneous.ENSG00000272512.1,Adipose_Subcutaneous.ENSG00000187608.10,Adipose_Subcutaneous.ENSG00000188157.16",chr1:995371:C:G:b38,0.54586709,"76.709872, 9.671171, 14.237908",cos7:y31_y34_y37,trait_shared
60,"Adipose_Subcutaneous.ENSG00000272512.1,Adipose_Subcutaneous.ENSG00000187608.10,Adipose_Subcutaneous.ENSG00000188157.16",chr1:995153:C:G:b38,0.18233783,"77.85543, 10.22310, 12.71399",cos7:y31_y34_y37,trait_shared
61,"Adipose_Subcutaneous.ENSG00000272512.1,Adipose_Subcutaneous.ENSG00000187608.10,Adipose_Subcutaneous.ENSG00000188157.16",chr1:995982:G:A:b38,0.16805050,"79.775155, 8.826491, 13.023182",cos7:y31_y34_y37,trait_shared
62,"Adipose_Subcutaneous.ENSG00000272512.1,Adipose_Subcutaneous.ENSG00000187608.10,Adipose_Subcutaneous.ENSG00000188157.16",chr1:995187:A:G:b38,0.06600336,"75.75045, 10.23273, 12.64426",cos7:y31_y34_y37,trait_shared
69,"Adipose_Subcutaneous.ENSG00000310526.1,Adipose_Subcutaneous.ENSG00000304860.1,Adipose_Subcutaneous.ENSG00000008130.15",chr1:634553:G:A:b38,0.82758384,"2.549748, 3.360490, 2.987133",cos9:y1_y61_y97,trait_shared
70,"Adipose_Subcutaneous.ENSG00000310526.1,Adipose_Subcutaneous.ENSG00000304860.1,Adipose_Subcutaneous.ENSG00000008130.15",chr1:625776:T:G:b38,0.13442579,"3.567115, 2.851089, 1.841342",cos9:y1_y61_y97,trait_shared
73,"Adipose_Subcutaneous.ENSG00000177757.3,Adipose_Subcutaneous.ENSG00000178821.13",chr1:727034:C:T:b38,0.81098733,"4.887385, 2.610897",cos11:y19_y102,trait_shared
74,"Adipose_Subcutaneous.ENSG00000177757.3,Adipose_Subcutaneous.ENSG00000178821.13",chr1:727477:G:A:b38,0.09223768,"3.416091, 2.724539",cos11:y19_y102,trait_shared
